In [1]:
from configparser import ConfigParser
import os
from pathlib import Path
import shutil

import tqdm
import vak

In [2]:
BIRDS = ['bl26lb16', 'gy6or6', 'or60yw70', 'gr41rd51']

CONFIGS_DIR = Path('../../src/configs/')
BF_CONFIGS = sorted(list(CONFIGS_DIR.glob('*BFSongRepository*ini')))


configs_by_bird = {
    bird: [bf_config for bf_config in BF_CONFIGS if bird in str(bf_config)][0]
    for bird in BIRDS
}

In [3]:
BFSongRepo = Path('~/Documents/data/BFSongRepository/').expanduser()

all_notmats = list(BFSongRepo.glob('*/*/*.not.mat'))
bird_date_dirs = set([notmat.parents[0] for notmat in all_notmats])

copy all .cbins with .not.mats into a sub-directory

In [10]:
# for bird_date_dir in bird_date_dirs:
#     has_notmat = bird_date_dir.joinpath('has_notmat')
#     has_notmat.mkdir(exist_ok=True)
#     notmats_this_date_dir = sorted(list(bird_date_dir.glob('*.not.mat')))
#     print(f'\ncopying annotated songs in {bird_date_dir} into sub-directory')
#     for notmat in tqdm.tqdm(notmats_this_date_dir):
#         shutil.copy(notmat, dst=has_notmat)
#         cbin = notmat.parent.joinpath(
#             Path(notmat.stem).stem
#         )
#         shutil.copy(cbin, dst=has_notmat)  # cbin_file, stem.stem removes .not.mat
#         rec = notmat.parent.joinpath(
#             Path(Path(notmat.stem).stem).stem + '.rec'
#         )
#         shutil.copy(rec, dst=has_notmat)
#         tmp = notmat.parent.joinpath(
#             Path(Path(notmat.stem).stem).stem + '.tmp'
#         )
#         shutil.copy(tmp, dst=has_notmat)

 20%|█▉        | 17/87 [00:00<00:00, 169.53it/s]


copying annotated songs in {bird_date_dir} into sub-directory


 16%|█▋        | 33/202 [00:00<00:00, 322.10it/s]


copying annotated songs in {bird_date_dir} into sub-directory


 18%|█▊        | 21/117 [00:00<00:00, 208.57it/s]

...

 11%|█         | 22/196 [00:00<00:00, 214.68it/s]


copying annotated songs in {bird_date_dir} into sub-directory


 36%|███▋      | 59/162 [00:00<00:00, 589.83it/s]


copying annotated songs in {bird_date_dir} into sub-directory


100%|██████████| 162/162 [00:00<00:00, 580.73it/s]


get dirs to predict for each bird

In [4]:
dirs_to_predict = {}
for bird in BIRDS:
    these = [
        bird_date_dir for bird_date_dir in bird_date_dirs
        if bird in str(bird_date_dir)
    ]
    these = [path.joinpath('has_notmat')
             for path in these]
    dirs_to_predict[bird] = these

In [5]:
spect_params = {'fft_size': 512,
                'step_size': 62,
                'freq_cutoffs': [500, 10000],
                'thresh': 6.25,
                'transform_type': 'log_spect'}
sp_nt = vak.config.spectrogram.SpectConfig(**spect_params)

In [6]:
for bird in BIRDS:
    print(f'predicting segments and labels for bird: {bird}')
    config_ini = configs_by_bird[bird]
    config_obj = ConfigParser()
    config_obj.read(config_ini)

    train_config = vak.config.train.parse_train_config(config_obj, config_ini)
    net_config = vak.config.parse._get_nets_config(config_obj, train_config.networks)

    results_dir = config_obj['OUTPUT']['results_dir_made_by_main_script']
    checkpoint_path = str(Path(results_dir).joinpath('TweetyNet'))
    spect_scaler_path = str(Path(results_dir).joinpath('spect_scaler'))

    # TODO: fix path
    print(f'\tgetting labelmap from {train_config.train_vds_path}')
    train_vds = vak.dataset.VocalizationDataset.load(train_config.train_vds_path)
    train_vds = train_vds.load_spects()
    labelmap = train_vds.labelmap
    
    bird_dirs_predict = dirs_to_predict[bird]    
    for dir_to_predict in bird_dirs_predict:
        vds_fname = str(dir_to_predict.joinpath('predict.vds.json'))
        print(f'\tmaking dataset from {dir_to_predict}')
        predict_vds = vak.dataset.prep(str(dir_to_predict),
                                       audio_format='cbin',
                                       spect_params=sp_nt,
                                       return_vds=True,
                                       return_path=False)
        predict_vds = predict_vds.clear_spects()
        predict_vds.save(json_fname=vds_fname)

        print(f'\trunning vak.core.predict on {dir_to_predict}')
        vak.core.predict(
            predict_vds_path=vds_fname,
            checkpoint_path=checkpoint_path,
            networks=net_config,
            labelmap=labelmap,
            spect_scaler_path=spect_scaler_path,
            save_predict_vds=True,
        )

predicting segments and labels for bird: bl26lb16
	getting labelmap from /home/nickledave/Documents/data/BFSongRepository/vak/bl26lb16/_prep_190701_205301.train.vds.json
[########################################] | 100% Completed |  0.5s
	making dataset from /home/nickledave/Documents/data/BFSongRepository/bl26lb16/042012/has_notmat
[########################################] | 100% Completed | 25.5s
[########################################] | 100% Completed | 21.8s
	running vak.core.predict on /home/nickledave/Documents/data/BFSongRepository/bl26lb16/042012/has_notmat
[########################################] | 100% Completed | 19.3s
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/

Predicting labels for ['/home/nickledave/Documents/data/BFSongRepository/bl26lb16/042012/has_notmat/predict.vds.json']: 100%|██████████| 3059/3059 [02:47<00:00, 18.21it/s]


> /home/nickledave/Documents/repos/vak/src/vak/utils/labels.py(309)lbl_tb2labels()
-> idx = np.diff(labeled_timebins, axis=0).astype(np.bool)


(Pdb)  np.diff(labeled_timebins, axis=0).astype(np.bool)


array([ True, False, False, ..., False, False, False])


...

*** SyntaxError: invalid syntax


(Pdb)  idx = np.diff(labeled_timebins, axis=0).astype(np.bool)
(Pdb)  idx2 = np.insert(idx, 0, True)
(Pdb)  idx2


array([ True,  True, False, ..., False, False, False])


(Pdb)  q


BdbQuit: 